In [1]:
import salabim as sim
import math

In [ ]:


# Constants
up = 1
down = -1
still = 0

move_time = 2
dooropen_time = 2
doorclose_time = 2

enter_time = 2
exit_time = 1

capacity = 8

topfloor = 12  # 0: Ground floor, 1-12: Employee floors

elevatorcount = 2 

id = "morning" #persons created in the morning

def run_simulation(seed: int, trace=True, sim_duration=10*3600, warmup=3600) -> list:
    env = sim.Environment(random_seed=seed)
    env.animation_parameters(animate=True, title="Elevator Simulation")
    
    class PersonGenerator(sim.Component):
        def setup(self, from_, to, id, *args, **kwargs):
            self.from_ = from_
            self.to = to
            self.id = id

        def process(self):
            while True:
                from_ = sim.random.randint(0, topfloor)
                while True:
                    to = sim.random.randint(0, topfloor)
                    if from_ != to:
                        break

                    iat = 3600 / load
                    r = sim.random.uniform(0.5, 1.5)
                    yield self.hold(r * iat)


    class Person(sim.Component):
        def setup(self, from_, to, *args, **kwargs):
            self.fromfloor = floors[from_]
            self.tofloor = floors[to]
            self.direction = getdirection(self.fromfloor, self.tofloor)
            self.animate = sim.AnimateCircle(x=0, y=from_ * 30 + 15, radius=10, fillcolor="green", text=f"P{self.name()}")
            self.animate.x = 10  # some arbitrary position on the left for the queue
            self.animate.y = self.fromfloor.n * 30 + 15
            

        def process(self):
            self.enter(self.fromfloor.Persons)
            if not (self.fromfloor, self.direction) in requests:
                requests[self.fromfloor, self.direction] = self.env.now()
            self.enter(waitingline) #put itself at tail of the waiting line
             # Choose the elevator with the fewest passengers
            if Elevator1.count_to_floor(self.tofloor) <= Elevator2.count_to_floor(self.tofloor):
                chosen_elevator = Elevator1
            else:
                chosen_elevator = Elevator2

            chosen_elevator.activate()
            self.passivate()

    class Elevator(sim.Component):
        def setup(self, capacity):
            self.capacity = capacity
            self.direction = still
            self.floor = floors[0]
            self.Persons = sim.Queue(name="Persons in Elevator")
            self.time_spent_moving = 0  # Initialize the time_spent_moving attribute
            self.last_movement_time = self.env.now()  # Initialize the last_movement_time attribute
            self.animate = sim.AnimateRectangle((250, 0, 450, 30), fillcolor="orange", text=f"Elevator {self.name()}")


        def process(self):
            while True:
                for direction, target_floor in [(up, topfloor), (down, 0)]:
                    for floor in range(self.floor.n, target_floor + direction, direction):
                        current_floor = floors[floor]

                        # Unload passengers if needed
                        if any(Person.tofloor == current_floor for Person in self.Persons):
                            self.hold(exit_time)
                            for Person in list(self.Persons):
                                if Person.tofloor == current_floor:
                                    Person.leave(self.Persons)
                                    Person.activate()
                                    Person.animate.x = 500  # some arbitrary position to represent the departure

                        # Load passengers if needed
                        if (current_floor, direction) in requests:
                            del requests[current_floor, direction]
                            self.hold(dooropen_time)

                            for Person in list(current_floor.Persons):
                                if len(self.Persons) < capacity:
                                    Person.leave(current_floor.Persons)
                                    self.person = waitingline.pop()
                                    Person.enter(self.Persons)
                                    Person.animate.x = 350  # inside the elevator
                                    Person.animate.y = self.floor.n * 30 + 15


                            self.hold(enter_time)
                            self.hold(doorclose_time)
                            
                        #Logic for choosing the elevator
                        
                        
                        # Move to the next floor
                        if floor != target_floor:
                            self.hold(move_time)
                        self.floor = floors[floor]
                        self.animate.y = self.floor.n * 30 + 15
                         # Update time spent moving at each tick
                        self.time_spent_moving += self.env.now() - self.last_movement_time
                        self.last_movement_time = self.env.now()
                        
                         #Generate staircase sequence if waiting line is too big
                        if waitingline.length() > 16 and target_floor - floor < 3:
                            Person.leave(current_floor.Persons)
                            self.person = waitingline.pop()
                
                # Account for dooropen_time and exit_time
                self.hold(dooropen_time + exit_time)
                self.time_spent_moving -= dooropen_time + exit_time
                



        def count_to_floor(self, tofloor):
            n = 0
            for Person in self.Persons:
                if Person.tofloor == tofloor:
                    n += 1
            return n


    class Floor:
        def __init__(self, n):
            self.n = n
            self.Persons = sim.Queue(name=f"Persons {n}")
            self.animate = sim.AnimateRectangle((0, n*30, 200, (n+1)*30), fillcolor="lightblue", text=f"Floor {n}")

    def getdirection(fromfloor, tofloor):
        return up if fromfloor.n < tofloor.n else down

    waitingline = sim.Queue("waitingline")
    requests = {}
    floors = {ifloor: Floor(ifloor) for ifloor in range(topfloor + 1)}
    #elevator = Elevator(capacity=capacity, at=0)
    

    # Initialize the floor and elevator instances
    requests = {}
    floors = {ifloor: Floor(ifloor) for ifloor in range(topfloor + 1)}
    Elevator1 = Elevator(capacity=capacity, at=0)
    Elevator2 = Elevator(capacity=capacity, at=0)
    
    # Create monitoring objects for time spent moving for each elevator
    elevator1_time_monitor = sim.Monitor(name="Elevator 1 Time Moving")
    elevator2_time_monitor = sim.Monitor(name="Elevator 2 Time Moving")
    
    # Generate 396 employees going from the ground floor to random employee floors
    for i in range(396):
        to_floor = sim.random.randint(0, topfloor)
        from_floor = sim.random.randint(0, topfloor)
        Person(from_=from_floor, to=to_floor, at=i*9)

    # Generate 80 random moves for the 8 working hours (10 moves per hour)
    for i in range(80):
        from_floor, to_floor = sim.random.randint(0, topfloor), sim.random.randint(0, topfloor)
        while from_floor == to_floor:
            from_floor, to_floor = sim.random.randint(0, topfloor), sim.random.randint(0, topfloor)
        Person(from_=from_floor, to=to_floor, at=3600 + i*450)
        
    #Generate 180 random customers 8 at working hours
    for i in range(180):
        to_floor = sim.random.randint(0, topfloor)
        from_floor = sim.random.randint(0, topfloor)
        Person(from_=from_floor, to=to_floor, at=3600+ i*160)
    
    env.run(duration=36000)  # Run for 10 hours (3600*10)

    # Enable monitoring after warmup
    waitingline.monitor(True)
    for floor in floors.values():
        floor.Persons.monitor(True)
    elevator1_time_monitor.monitor(True)
    elevator2_time_monitor.monitor(True)

    env.run(duration=sim_duration)
    
     # Collect statistics for time spent moving for each elevator
    elevator1_time_monitor.tally(Elevator1.time_spent_moving)
    elevator2_time_monitor.tally(Elevator2.time_spent_moving)

    monitored = [waitingline] + [floor.Persons for floor in floors.values()]
    monitored += [elevator1_time_monitor, elevator2_time_monitor]
    return {queue.name(): queue for queue in monitored}

if __name__ == "__main__":
    res = run_simulation(seed=42, trace=True, sim_duration=36000, warmup=3600)
    for name, queue in res.items():
        print("--------------------------")
        queue.print_statistics()



   54+     18.000 person.2             current                              
   55                                  person.2                             enter Persons 3
   58                                  person.2                             enter waitingline
   65                                  elevator.0 activate                  scheduled for 18.000 @  116+
   66                                  person.2 passivate                   @   66+
  116+     18.000 elevator.1           current                              
  116                                  elevator.1 hold +2.000               scheduled for 20.000 @  116+
  116+     18.000 elevator.0           current                              
  116                                  elevator.0 hold +2.000               scheduled for 20.000 @  116+
  116+     20.000 elevator.1           current                              
  116                                  elevator.1 hold +2.000               scheduled for 22.000 @  116+
  